In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import math
from matplotlib.gridspec import GridSpec
from numpy import mean

####For site 1:
sites = 'US-Ha1'

df=pd.read_csv('D:\Onedrive\OneDrive - University of Oklahoma\Documents\PHD\Class\Class22Fall\Python2022\Homework\project\AMF_US-Ha1_FLUXNET_SUBSET_DD_1991-2020_3-5.csv')
vi=pd.read_csv('D:\Onedrive\OneDrive - University of Oklahoma\Documents\PHD\Class\Class22Fall\Python2022\Homework\project\VI_usha1.csv')
modis=pd.read_csv('D:\Onedrive\OneDrive - University of Oklahoma\Documents\PHD\Class\Class22Fall\Python2022\Homework\project\MOD17_usha1.csv')
TA_F = df['TA_F'].values.tolist()
PPFD_IN = df['PPFD_IN']
PPFD_IN[PPFD_IN<0] = np.nan
PPFD_IN = PPFD_IN.values.tolist()
GPP_ec = df['GPP_DT_VUT_REF'].values.tolist()
GPPmod17a2h = modis['Gpp'].values.tolist()
#print('GPPmod17a2h',len(GPPmod17a2h))
#print('GPPmod17a2h',GPPmod17a2h)



evi = vi['evi']
evi = evi.values.tolist()
lswi = vi['lswi']
lswi = lswi.values.tolist()
#print(evi)
#print(len(evi))
#print(lswi)
#print(len(lswi))


#for row in range(2,TA_F.max_row+1):
   
#arr = [] 
#arr.append(np.mean(TA_F[0:8]))
#print(arr)
#print(TA_F[8:16])

year = int(len(TA_F)/365)
interval_numbers = math.ceil((len(TA_F)/8))
print('interval_numbers: ',interval_numbers)


TA = []
for i in range(interval_numbers):
    TA.append(np.nanmean(TA_F[i*8:i*8+8]))
#print(TA)
#print(PPFD_IN)
#for value in PPFD_IN:
#    if value > 0:
#        print(value)
#        PPFD_IN.remove(value)
#print(PPFD_IN)



PAR = []
for i in range(interval_numbers):
    mean = np.nanmean(PPFD_IN[i*8:i*8+8]) * 48 * 1800 / 1000000
    PAR.append(mean)
#print(PAR)
#print(len(PAR))

GPPec = []
for i in range(interval_numbers):
    mean = np.nanmean(GPP_ec[i*8:i*8+8])
    GPPec.append(mean)
#print(GPPec)
#print(len(GPPec))


n = 0
Topt = []
while n < interval_numbers/46:
    y1 = 0 + 46 * n
    y2 = 46 + 46 * n
    zip_evi_TA = zip(evi[y1:y2],TA[y1:y2])
    sorted_zip = sorted(zip_evi_TA, key=lambda x:x[0])
    sorted_evi, sorted_TA = zip(*sorted_zip)
    #print(sorted_TA)
    opt = np.nanmean(sorted_TA[-3:])
    for p in range(46):
        Topt.append(opt)
    n += 1
#print(Topt)    



Tmin = [-1] * interval_numbers
Topt = Topt
Tmax = [40] * interval_numbers


T_scalar = []
for i in range(interval_numbers):
    va1 = TA[i] - Tmin[i]
    va2 = TA[i] - Tmax[i]
    va3 = TA[i] - Topt[i]
    
    va4 = va1 * va2
    va5 = va3 * va3
    
    va6 = va4/(va4-va5)
    T_scalar.append(va6)
    
#print(T_scalar)
#print(len(T_scalar))

W_scalar = []
lswi_max = []
j = 0
while j < interval_numbers/46:
    x1 = 12 + 46 * j
    x2 = 39 + 46 * j
    max_lswi = np.nanmax(lswi[x1:x2])
    for p in range(46):
        lswi_max.append(max_lswi)
    j += 1
#print(lswi_max)    

for i in range(interval_numbers):
    W_scalar.append((1+lswi[i])/(1+lswi_max[i]))
#print(W_scalar)

e0 = [0.42] * interval_numbers



eg = []
for i in range(interval_numbers):
    eg.append(e0[i] * T_scalar[i] * W_scalar[i])
#print(eg)
#print(len(eg))

APAR = []
for i in range(interval_numbers):
    fPAR = (evi[i]-0.1) * 1.25
    APAR.append(fPAR * PAR[i])
#print(APAR)
#print(len(APAR))

GPPvpm = []
for i in range(interval_numbers):
    GPPvpm.append(eg[i] * APAR[i])
#print(GPPvpm)













#####################################################################################
###   visualization
#####################################################################################


fig1 = plt.figure(facecolor='white')
fontsize = 15
ax = plt.subplot(111)

x = list(range(1,interval_numbers+1))
y1 = GPPec
y2 = GPPvpm
y3 = GPPmod17a2h
ax.plot(x, y1, ls = '-',lw = 2, c = '#1E90FF', label = 'GPP$_\mathrm{ec}$',alpha= 0.75,marker = 'o', ms = 5, markerfacecolor = 'white',zorder = 4)
ax.plot(x, y2, ls = '-',lw = 2, c = '#228B22', label = 'GPP$_\mathrm{vpm}$',alpha= 0.75,marker = 'o', ms = 5, markerfacecolor = 'white',zorder = 5)
ax.plot(x, y3, ls = '-',lw = 2, c = '#FF7F00', label = 'GPP$_\mathrm{MOD17A2H}$',alpha= 0.75,marker = 'o', ms = 5, markerfacecolor = 'white',zorder = 3)
lines,labels = ax.get_legend_handles_labels()
ax.legend(lines,labels,fontsize=fontsize,loc='upper left',frameon=False, ncol = 4, columnspacing=0.5, handletextpad=0.2)
plt.xlim(0,interval_numbers)
ax.set_ylim(0,23)


x_major_locator=MultipleLocator(46)
ax.xaxis.set_major_locator(x_major_locator)
ax.set_xticks([0,230,460,690,920])
ax.set_xticklabels([2000,2005,2010,2015,2020],fontsize=fontsize,rotation=0)
ax.set_yticks([0,5,10,15,20])
ax.set_yticklabels([0,5,10,15,20],fontsize=fontsize,rotation=0)
#ax.set_xlabel('Year',fontsize=fontsize)
ax.set_ylabel('GPP (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax.text(870, 21, sites,fontsize=fontsize) 
fig1.set_size_inches(12, 4)
plt.rcParams['figure.dpi'] = 600


fig2 = plt.figure(facecolor='white')
fontsize = 15
ax2 = plt.subplot(111)


new_y2=[]
for elem in y2:
	if not np.isnan(elem):
		new_y2.append(elem)

new_y1_1=[]
for i in range(interval_numbers):
    if not np.isnan(y2[i]):
        new_y1_1.append(y1[i])

new_y3=[]
for elem in y3:
	if not np.isnan(elem):
		new_y3.append(elem)

new_y1_2=[]
for i in range(interval_numbers):
    if not np.isnan(y3[i]):
        new_y1_2.append(y1[i])

def R_Square(x,y):
    p1=x2=y2=0.0
    x_=np.mean(x)
    y_=np.mean(y)
    for i in range(len(x)):
        p1+=(x[i]-x_)*(y[i]-y_)
        x2+=(x[i]-x_)**2
        y2+=(y[i]-y_)**2
    r=p1/((x2** 0.5)*(y2** 0.5))
    return r

ax2.scatter(y1, y2,c='#1E90FF', s=40,cmap='Spectral',alpha = 0.4,label = 'GPP$_\mathrm{ec}$  vs. GPP$_\mathrm{vpm}$')
ax2.plot((0, 20), (0, 20), ls='--',c='black')
ax2.set_xlim(0,20)
ax2.set_ylim(0,20)
z1 = np.polyfit(new_y1_1, new_y2, 1)
z1_new = []
for va in z1:
    z1_new.append(round(va,2))
p1 = np.poly1d(z1_new)
yvals1=p1(new_y1_1)
plot=ax2.plot(new_y1_1, yvals1, alpha = 0.8,c = 'red')
p1 = ['y = ', str(p1).replace('\n', '').replace('\r', '')]
a = ''
p1 = a.join(p1)
correlation1 = str(round(R_Square(new_y1_1,new_y2),2)).replace('\n', '').replace('\r', '')
ax2.set_xlabel('GPP$_\mathrm{ec}$ (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax2.set_ylabel('GPP$_\mathrm{vpm}$ (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax2.set_xticks([0,5,10,15,20])
ax2.set_yticks([0,5,10,15,20])
ax2.set_xticklabels([0,5,10,15,20],fontsize=fontsize,rotation=0)
ax2.set_yticklabels([0,5,10,15,20],fontsize=fontsize,rotation=0)
lines2,labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines2,labels2,fontsize=fontsize,loc='upper right',frameon=True, ncol = 4, columnspacing=0.5, handletextpad=0.2)
ax2.text(11, 4.8, p1,fontsize=fontsize) 
ax2.text(11, 2.8, "R$^\mathrm{2}$ = " + correlation1,fontsize=fontsize) 
ax2.text(11, 0.8, "N = " + str(len(new_y1_1)),fontsize=fontsize) 
fig2.set_size_inches(6, 6)
plt.rcParams['figure.dpi'] = 600

fig3 = plt.figure(facecolor='white')
fontsize = 15
ax3 = plt.subplot(111)

ax3.scatter(y1, y3,c='#228B22', s=40,cmap='Spectral',alpha = 0.4,label = 'GPP$_\mathrm{ec}$  vs. GPP$_\mathrm{MOD17A2H}$')
ax3.plot((0, 20), (0, 20), ls='--',c='black')
ax3.set_xlim(0,20)
ax3.set_ylim(0,20)
z2 = np.polyfit(new_y1_2, new_y3, 1)
z2_new = []
for va in z2:
    z2_new.append(round(va,2))
p2 = np.poly1d(z2_new)
yvals2=p2(new_y1_2)
p2 = ['y = ', str(p2).replace('\n', '').replace('\r', '')]
p2 = a.join(p2).rstrip('\n')
correlation2 = str(round(R_Square(new_y1_2,new_y3),2)).replace('\n', '').replace('\r', '')
plot=ax3.plot(new_y1_2, yvals2, alpha = 0.8,c = 'red')
ax3.set_xlabel('GPP$_\mathrm{ec}$ (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax3.set_ylabel('GPP$_\mathrm{MOD17A2H}$ (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax3.set_xticks([0,5,10,15,20])
ax3.set_yticks([0,5,10,15,20])
ax3.set_xticklabels([0,5,10,15,20],fontsize=fontsize,rotation=0)
ax3.set_yticklabels([0,5,10,15,20],fontsize=fontsize,rotation=0)
lines3,labels3 = ax3.get_legend_handles_labels()
ax3.legend(lines3,labels3,fontsize=fontsize,loc='upper right',frameon=True, ncol = 4, columnspacing=0.5, handletextpad=0.2)
ax3.text(11, 4.8, p2,fontsize=fontsize) 
ax3.text(11, 2.8, "R$^\mathrm{2}$ =" + correlation2 ,fontsize=fontsize) 
ax3.text(11, 0.8, "N = " + str(len(new_y1_2)),fontsize=fontsize) 
fig3.set_size_inches(6, 6)
plt.rcParams['figure.dpi'] = 600


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import math
from matplotlib.gridspec import GridSpec
from numpy import mean

####For site 1:
sites = 'US-Ton'

df=pd.read_csv('D:\Onedrive\OneDrive - University of Oklahoma\Documents\PHD\Class\Class22Fall\Python2022\Homework\project\FLX_US-Ton_FLUXNET2015_SUBSET_DD_2001-2014_1-4.csv')
vi=pd.read_csv('D:\Onedrive\OneDrive - University of Oklahoma\Documents\PHD\Class\Class22Fall\Python2022\Homework\project\VI_uston.csv')
modis=pd.read_csv('D:\Onedrive\OneDrive - University of Oklahoma\Documents\PHD\Class\Class22Fall\Python2022\Homework\project\MOD17_uston.csv')
TA_F = df['TA_F'].values.tolist()
PPFD_IN = df['PPFD_IN']
PPFD_IN[PPFD_IN<0] = np.nan
PPFD_IN = PPFD_IN.values.tolist()
GPP_ec = df['GPP_DT_VUT_REF'].values.tolist()
GPPmod17a2h = modis['Gpp'].values.tolist()
#print('GPPmod17a2h',len(GPPmod17a2h))
#print('GPPmod17a2h',GPPmod17a2h)



evi = vi['evi']
evi = evi.values.tolist()
lswi = vi['lswi']
lswi = lswi.values.tolist()
#print(evi)
#print(len(evi))
#print(lswi)
#print(len(lswi))


#for row in range(2,TA_F.max_row+1):
   
#arr = [] 
#arr.append(np.mean(TA_F[0:8]))
#print(arr)
#print(TA_F[8:16])

year = int(len(TA_F)/365)
interval_numbers = math.ceil((len(TA_F)/8))
print('interval_numbers: ',interval_numbers)


TA = []
for i in range(interval_numbers):
    TA.append(np.nanmean(TA_F[i*8:i*8+8]))
#print(TA)
#print(PPFD_IN)
#for value in PPFD_IN:
#    if value > 0:
#        print(value)
#        PPFD_IN.remove(value)
#print(PPFD_IN)



PAR = []
for i in range(interval_numbers):
    mean = np.nanmean(PPFD_IN[i*8:i*8+8]) * 48 * 1800 / 1000000
    PAR.append(mean)
#print(PAR)
#print(len(PAR))

GPPec = []
for i in range(interval_numbers):
    mean = np.nanmean(GPP_ec[i*8:i*8+8])
    GPPec.append(mean)
#print(GPPec)
#print(len(GPPec))


n = 0
Topt = []
while n < interval_numbers/46:
    y1 = 0 + 46 * n
    y2 = 46 + 46 * n
    zip_evi_TA = zip(evi[y1:y2],TA[y1:y2])
    sorted_zip = sorted(zip_evi_TA, key=lambda x:x[0])
    sorted_evi, sorted_TA = zip(*sorted_zip)
    #print(sorted_TA)
    opt = np.nanmean(sorted_TA[-3:])
    for p in range(46):
        Topt.append(opt)
    n += 1
#print(Topt)    



Tmin = [1] * interval_numbers
Topt = Topt
Tmax = [48] * interval_numbers


T_scalar = []
for i in range(interval_numbers):
    va1 = TA[i] - Tmin[i]
    va2 = TA[i] - Tmax[i]
    va3 = TA[i] - Topt[i]
    
    va4 = va1 * va2
    va5 = va3 * va3
    
    va6 = va4/(va4-va5)
    T_scalar.append(va6)
    
#print(T_scalar)
#print(len(T_scalar))

W_scalar = []
lswi_max = []
j = 0
while j < interval_numbers/46:
    x1 = 12 + 46 * j
    x2 = 39 + 46 * j
    max_lswi = np.nanmax(lswi[x1:x2])
    for p in range(46):
        lswi_max.append(max_lswi)
    j += 1
#print(lswi_max)    

for i in range(interval_numbers):
    W_scalar.append((1+lswi[i])/(1+lswi_max[i]))
#print(W_scalar)

e0 = [0.42] * interval_numbers



eg = []
for i in range(interval_numbers):
    eg.append(e0[i] * T_scalar[i] * W_scalar[i])
#print(eg)
#print(len(eg))

APAR = []
for i in range(interval_numbers):
    fPAR = (evi[i]-0.1) * 1.25
    APAR.append(fPAR * PAR[i])
#print(APAR)
#print(len(APAR))

GPPvpm = []
for i in range(interval_numbers):
    GPPvpm.append(eg[i] * APAR[i])
#print(GPPvpm)













#####################################################################################
###   visualization
#####################################################################################


fig1 = plt.figure(facecolor='white')
fontsize = 15
ax = plt.subplot(111)

x = list(range(1,interval_numbers+1))
y1 = GPPec
y2 = GPPvpm
y3 = GPPmod17a2h
ax.plot(x, y1, ls = '-',lw = 2, c = '#1E90FF', label = 'GPP$_\mathrm{ec}$',alpha= 0.75,marker = 'o', ms = 5, markerfacecolor = 'white',zorder = 4)
ax.plot(x, y2, ls = '-',lw = 2, c = '#228B22', label = 'GPP$_\mathrm{vpm}$',alpha= 0.75,marker = 'o', ms = 5, markerfacecolor = 'white',zorder = 5)
ax.plot(x, y3, ls = '-',lw = 2, c = '#FF7F00', label = 'GPP$_\mathrm{MOD17A2H}$',alpha= 0.75,marker = 'o', ms = 5, markerfacecolor = 'white',zorder = 3)
lines,labels = ax.get_legend_handles_labels()
ax.legend(lines,labels,fontsize=fontsize,loc='upper left',frameon=False, ncol = 4, columnspacing=0.5, handletextpad=0.2)
plt.xlim(0,interval_numbers)
ax.set_ylim(0,10)


x_major_locator=MultipleLocator(46)
ax.xaxis.set_major_locator(x_major_locator)
ax.set_xticks([0,138,276,414,552,644])
ax.set_xticklabels([2001,2004,2007,2010,2013,2015],fontsize=fontsize,rotation=0)
ax.set_yticks([0,5,10])
ax.set_yticklabels([0,5,10],fontsize=fontsize,rotation=0)
#ax.set_xlabel('Year',fontsize=fontsize)
ax.set_ylabel('GPP (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax.text(590, 9, sites,fontsize=fontsize) 
fig1.set_size_inches(12, 4)
plt.rcParams['figure.dpi'] = 600


fig2 = plt.figure(facecolor='white')
fontsize = 15
ax2 = plt.subplot(111)


new_y2=[]
for elem in y2:
	if not np.isnan(elem):
		new_y2.append(elem)

new_y1_1=[]
for i in range(interval_numbers):
    if not np.isnan(y2[i]):
        new_y1_1.append(y1[i])

new_y3=[]
for elem in y3:
	if not np.isnan(elem):
		new_y3.append(elem)

new_y1_2=[]
for i in range(interval_numbers):
    if not np.isnan(y3[i]):
        new_y1_2.append(y1[i])

def R_Square(x,y):
    p1=x2=y2=0.0
    x_=np.mean(x)
    y_=np.mean(y)
    for i in range(len(x)):
        p1+=(x[i]-x_)*(y[i]-y_)
        x2+=(x[i]-x_)**2
        y2+=(y[i]-y_)**2
    r=p1/((x2** 0.5)*(y2** 0.5))
    return r

ax2.scatter(y1, y2,c='#1E90FF', s=40,cmap='Spectral',alpha = 0.4,label = 'GPP$_\mathrm{ec}$  vs. GPP$_\mathrm{vpm}$')
ax2.plot((0, 10), (0, 10), ls='--',c='black')
ax2.set_xlim(0,10)
ax2.set_ylim(0,10)
z1 = np.polyfit(new_y1_1, new_y2, 1)
z1_new = []
for va in z1:
    z1_new.append(round(va,2))
p1 = np.poly1d(z1_new)
yvals1=p1(new_y1_1)
plot=ax2.plot(new_y1_1, yvals1, alpha = 0.8,c = 'red')
p1 = ['y = ', str(p1).replace('\n', '').replace('\r', '')]
a = ''
p1 = a.join(p1)
correlation1 = str(round(R_Square(new_y1_1,new_y2),2)).replace('\n', '').replace('\r', '')
ax2.set_xlabel('GPP$_\mathrm{ec}$ (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax2.set_ylabel('GPP$_\mathrm{vpm}$ (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax2.set_xticks([0,5,10])
ax2.set_yticks([0,5,10])
ax2.set_xticklabels([0,5,10],fontsize=fontsize,rotation=0)
ax2.set_yticklabels([0,5,10],fontsize=fontsize,rotation=0)
lines2,labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines2,labels2,fontsize=fontsize,loc='upper right',frameon=True, ncol = 4, columnspacing=0.5, handletextpad=0.2)
ax2.text(5.5, 2.8, p1,fontsize=fontsize) 
ax2.text(5.5, 1.8, "R$^\mathrm{2}$ = " + correlation1,fontsize=fontsize) 
ax2.text(5.5, 0.8, "N = " + str(len(new_y1_1)),fontsize=fontsize) 
fig2.set_size_inches(6, 6)
plt.rcParams['figure.dpi'] = 600

fig3 = plt.figure(facecolor='white')
fontsize = 15
ax3 = plt.subplot(111)

ax3.scatter(y1, y3,c='#228B22', s=40,cmap='Spectral',alpha = 0.4,label = 'GPP$_\mathrm{ec}$  vs. GPP$_\mathrm{MOD17A2H}$')
ax3.plot((0, 10), (0, 10), ls='--',c='black')
ax3.set_xlim(0,10)
ax3.set_ylim(0,10)
z2 = np.polyfit(new_y1_2, new_y3, 1)
z2_new = []
for va in z2:
    z2_new.append(round(va,2))
p2 = np.poly1d(z2_new)
yvals2=p2(new_y1_2)
p2 = ['y = ', str(p2).replace('\n', '').replace('\r', '')]
p2 = a.join(p2).rstrip('\n')
correlation2 = str(round(R_Square(new_y1_2,new_y3),2)).replace('\n', '').replace('\r', '')
plot=ax3.plot(new_y1_2, yvals2, alpha = 0.8,c = 'red')
ax3.set_xlabel('GPP$_\mathrm{ec}$ (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax3.set_ylabel('GPP$_\mathrm{MOD17A2H}$ (gC m$^\mathrm{-2}$day$^\mathrm{-1}$)',fontsize=fontsize)
ax3.set_xticks([0,5,10])
ax3.set_yticks([0,5,10])
ax3.set_xticklabels([0,5,10],fontsize=fontsize,rotation=0)
ax3.set_yticklabels([0,5,10],fontsize=fontsize,rotation=0)
lines3,labels3 = ax3.get_legend_handles_labels()
ax3.legend(lines3,labels3,fontsize=fontsize,loc='upper right',frameon=True, ncol = 4, columnspacing=0.5, handletextpad=0.2)
ax3.text(5.5, 2.8, p2,fontsize=fontsize) 
ax3.text(5.5, 1.8, "R$^\mathrm{2}$ =" + correlation2 ,fontsize=fontsize) 
ax3.text(5.5, 0.8, "N = " + str(len(new_y1_2)),fontsize=fontsize) 
fig3.set_size_inches(6, 6)
plt.rcParams['figure.dpi'] = 600
